# Consultas en Mongo

Busco al 'mejor'  superhéroe según su calificación en 'Powerstats'. Los criterios evaluados son:
- Intelligence
- Strength
- Speed
- Durability
- Power
- Combat

In [ ]:
db.supers.aggregate([
...   {
    $project: {
...       name: 1,
...       totalScore: {
...         $sum: [
...           "$powerstats.intelligence",
...           "$powerstats.strength",
...           "$powerstats.speed",
...           "$powerstats.durability",
...           "$powerstats.power",
...           "$powerstats.combat"
...         ]
...       }
...     }
...   },
...   {
...     $sort: { totalScore: -1 }
...   },
...   {
...     $limit: 1
...   }
... ])
[

Ahora ponderamos los poderes de los superhéroes
- Inteligencia : 5
- Fuerza : 4
- Velociodad : 3
- Durabilidad: 2
- Poder: 4
- Combate: 3 

In [ ]:
db.supers.aggregate([
  {
    $project: {
      name: 1,
      totalScore: {
        $sum: [
          { $multiply: ["$powerstats.intelligence", 5] },
          { $multiply: ["$powerstats.strength", 4] },
          { $multiply: ["$powerstats.speed", 3] },
          { $multiply: ["$powerstats.durability", 2] },
          { $multiply: ["$powerstats.power", 4] },
          { $multiply: ["$powerstats.combat", 3] }
        ]
      }
    }
  },
  {
    $sort: { totalScore: -1 }
  },
  {
    $limit: 1
  }
])


Recupero las ocupaciones de los hombres

In [ ]:
db.supers.aggregate([
...   {
...     $match: {
...       "appearance.gender": "Male"
...     }
...   },
...   {
...     $project: {
...       _id: 0,
...       occupation: "$work.occupation"
...     }
...   },
...   {
...     $unwind: "$occupation"
...   },
...   {
...     $group: {
...       _id: "$occupation",
...       count: { $sum: 1 }
...     }
...   },
...   {
...     $sort: { count: -1 }
...   }
... ])

Agrupa la cantidad de super héroes cuyas actividades está incluidos: 
- Aventurero
- Profesor
- Ingeniero
- Estudiante



In [ ]:
db.supers.aggregate([
  {
    $match: {
      "work.occupation": { $exists: true }
    }
  },
  {
    $project: {
      _id: 0,
      occupation: "$work.occupation"
    }
  },
  {
    $unwind: "$occupation"
  },
  {
    $group: {
      _id: {
        $cond: {
          if: {
            $regexMatch: {
              input: "$occupation",
              regex: /(Adventurer|Professor|Engineer|Student|Scientist)/i
            }
          },
          then: {
            $regexFind: {
              input: "$occupation",
              regex: /(Adventurer|Professor|Engineer|Student|Scientist)/i
            }
          },
          else: "Other"
        }
      },
      count: { $sum: 1 }
    }
  },
  {
    $sort: { count: -1 }
  }
])


Las mujeres de DC cuya inteligencia y poder de combate sea mayor a la media.

In [ ]:
db.supers.aggregate([
  {
    $match: {
      "biography.publisher": "DC Comics",
      "appearance.gender": "Female"
    }
  },
  {
    $group: {
      _id: "$name",
      averageIntelligence: { $avg: "$powerstats.intelligence" },
      averageCombat: { $avg: "$powerstats.combat" }
    }
  },
  {
    $project: {
      _id: 1,
      averageIntelligence: 1,
      averageCombat: 1
    }
  }
])

   